# Personal Search 03

Busca de informações em livros usando FAISS

## Import packages

In [1]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_community.llms import Ollama
from IPython.display import display, Markdown
import logging
from langchain import hub
import os
#logging.basicConfig(level=logging.INFO)

In [2]:
# limpar memória gpu
#import torch
#torch.cuda.empty_cache()

## Configuration

In [3]:
model = "llama3:8b"
#model = "mistral:7b"
#model = "phi3"
embeddings_model = "nomic-embed-text:v1.5"
path_books = "./vector-store/books"
question = "O que falta aos pregadores de hoje?"
LANGCHAIN_HUB_API_KEY = os.getenv("LANGCHAIN_HUB_API_KEY")

## Retriever Instance

In [4]:
embeddings = OllamaEmbeddings(model=embeddings_model)

db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = False
retriever.search_kwargs["k"] = 2

## Search

In [5]:
sources = retriever.invoke(question)
livros = ''
for source in sources:
    source_name = source.metadata['source'].split('/')[-1]
    source_page = source.metadata['page']
    livros += f'**Fonte:** {source_name} - Página: {source_page}\n\n'
    livros += f'{source.page_content}\n\n'

In [6]:
# RAG
template = """
Cite trechos do contexto que tenha relação com "{question}". Não elabore, apenas cite.
Caso não encontre, apenas diga que não encontrou. 
Obedeça o formato "Citação do livro" - Fonte: Nome do Livro - Página: Número da Página

Contexto:
{context}

Responda em português.
"""

prompt = ChatPromptTemplate.from_template(template)
#hub.push(repo_full_name="personal-search/rag-personal-search-03", object=prompt, api_key=LANGCHAIN_HUB_API_KEY)

llm = Ollama(model=model, temperature=0)
#llm = ChatOllama(model=model, temperature=0)

final_rag_chain = (
    {"context": itemgetter("livros"), 
     "question": itemgetter("question")} 
    | prompt
    | llm.bind(stop=["<|eot_id|>"])
    | StrOutputParser()
)
# llm.bind(stop=["<|eot_id|>"])

final_answer = final_rag_chain.invoke({"question":question, "livros":livros})

In [7]:
display(Markdown(f"**Tópico:** {question}\n\n"))
display(Markdown(final_answer))

**Tópico:** O que falta aos pregadores de hoje?



**Fonte:** Conselhos para obreiros - Charles H Spurgeon.pdf - Página: 38-40

O que falta aos pregadores de hoje é a determinação e a coragem para se aproximar do povo, como os ceifeiros de trigo. Eles devem sentir-se um com o povo e amá-las, relacionar-se com elas, sentir prazer em vê-las. Além disso, eles devem ser honestos e francos ao pregar, chamando o pecado de pecado e alertando as pessoas para a necessidade de escapar da ira divina pela fé em Jesus.

**Fonte:** Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf - Página: 72-74

O que falta aos pregadores de hoje é a coragem para se aproximar do povo e criticá-lo, como os profetas do passado. Eles devem ser mortos por causa da indignação santa ao ver a religião falsa enganando as pessoas e roubando-lhes a salvação. Além disso, eles devem ser pregadores que possam incendiar o coração dos homens e não poupem ninguém, clamando em alta voz e não se preocupando com a perseguição.

In [8]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 38}

**Trecho:** Afinal de contas, que tipo de obreiros são solicitados? Devem ser homens que vão até o campo.
Não há como colher o trigo ficando a certa distância da plantação e acenando para ela: você tem
que ir até o pé de trigo; todo ceifeiro sabe disso. E você não conseguirá mover o coração das
pessoas e levá-las a Cristo imaginando-se um ser superior que faz uma concessão magnânima
quando cumprimenta um pobre. Essa forma distanciada de pregação é tão ridícula quanto ceifar
com uma faca de mesa com cabo de marfim e usando luvas de pelica; eu não creio que Deus
abençoe esse trabalho. Embrenhem-se no campo de trigo, como homens determinados! Os servos
de Deus deveriam sentir que são um com o povo; sejam quem forem as pessoas, os servos
deveriam amá-las, relacionar-se com elas, sentir prazer em vê-las. Olhar para essas pessoas e
dizer: “irmão”. Todo homem é meu irmão, mesmo o mau, porém eu o amarei e o levarei até
Jesus. Os ceifeiros de Cristo devem descer até o trigo.
Agora, observe o que o trabalhador traz consigo: uma foice. Sua comunicação com a espiga de
trigo é rápida e afiada. Ele a corta; corta, e a espiga cai ao chão. O homem que Deus diz ser um
trabalhador da seara não pode usar de palavras suaves e delicadas, persuadir com doutrinas que
falam da dignidade humana, da excelência da autoajuda, do esforço próprio para corrigir a
condição pecaminosa e coisas semelhantes. Deus pode amaldiçoar tal hipócrita, pois ele é a
maldição deste século. O pregador honesto chama o pecado de pecado e fala francamente: “Você
está se arruinando; enquanto rejeitar Cristo, viverá nas bordas do inferno e brevemente estará
perdido por toda a eternidade. Não há discussão: você tem de escapar da ira divina pela fé em
Jesus, ou será tirado eternamente da presença de Deus e de toda esperança de alegria”. O sermão
do pregador deve ser direto. Ele não tem de cegar a navalha da segadeira temendo machucar
alguém. O evangelho pretende ferir a consciência e ir direto ao coração a fim de separar a alma
do pecado e do ego, da mesma maneira que o grão é separado do solo. Nosso objetivo é cortar
imediatamente o pecado, pois toda a beleza da carne deve ser derrotada; toda a sua glória e
excelência devem ser extirpadas. E o homem deve surgir como um morto para que seja salvo. Os
ministros que não têm o objetivo de cortar profundamente o pecado não são sal da terra. Deus
nunca enviaria homens que não têm como objetivo incomodar a consciência humana. Tal homem

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 72}

**Trecho:** O mesmo havia acontecido a Sa vonarola, que abalou a cidade de 
Florença, na Itália, a ponto de o rosto desse “monge louco” tornar -se 
um terror para os florentinos, e motivo de chacota para os fanáticos 
papistas.  
Irmãos, à luz do conhecimento que temos sobre o altar de Deus, é 
melhor vi vermos seis meses com o coração em chamas, apontando o 
pecado deste mundo, seja em que lugar for, e conclamando o povo a 
libertar -se do poder de Satanás e se voltar para Deus (como fez João 
Batista), do que morrer cercado de honrarias eclesiásticas e de 
doutorados em teologia, para se tornar motivo de riso no inferno, para 
os espíritos das trevas. Ridicularizar os magnatas da bebida e 
censurar os políticos corruptos não atrai maldição sobre a nossa 
cabeça. Há quem faça as duas coisas sem sofrer nenhuma amea ça à 
sua vida e à sua posição no púlpito. Os profetas do passado eram 
mortos po rque combatiam veementemente as religiões falsas. E nós 
também devíamos nos deixar arder de santa indignação ao ver a 
religião falsa enganando nossos semelhantes, e roubando de nossos 
entes queridos a salvação; ou ao ver sacerdotes levando -os para o 
inferno sob a efígie de um crucifixo. E talvez, quem sabe, daqui a 
alguns anos, p ara abrir o caminho para uma nova reforma no século 
XX, nós sejamos queimados em fogueiras.  
Esta aqui é para se ler e chorar: “Hoje o protestantismo mutilado vê 
os sacerdotes católicos romanos elogiando os evangelistas 
protestantes”. Respond a em sã consciência, você acredita que esses 
mesmos papistas aplaudiriam Lutero ou apoiariam Savo naro la? Ó 
Deus, envi a-nos pregadores que possam entre gar mensagens que 
penetrem o coração dos homens e o incendeie! Envia -nos uma 
geração de pregadores mártires, de homens em chamas, 
quebrantados e prostrados diante da visão de um castigo iminente e 
de um inferno eterno para os irregenerados!  
Que Deus nos envie profetas, homens destemidos que clamem em 
alta voz e não poupem ninguém, que  abalem nações com lamentos 
ungidos, que sejam fer vorosos quase a ponto de se tornarem 
insuportáveis, duros a ponto de ser difícil ouvi -los, e 
descomprome tidos a ponto de sofrerem perseguição. Esta mos 
cansados de pregadores que se apresentam de roupas elegantes, 
linguagem suave e torrentes de palavras, mas apenas com uma gota 
de unção. São homens que entendem mais de competição do que 
consagração , mais de promoção d o que oração. Substituem o

In [9]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))

**Fonte:** Afinal de contas, que tipo de obreiros são solicitados? Devem ser homens que vão até o campo.
Não há como colher o trigo ficando a certa distância da plantação e acenando para ela: você tem
que ir até o pé de trigo; todo ceifeiro sabe disso. E você não conseguirá mover o coração das
pessoas e levá-las a Cristo imaginando-se um ser superior que faz uma concessão magnânima
quando cumprimenta um pobre. Essa forma distanciada de pregação é tão ridícula quanto ceifar
com uma faca de mesa com cabo de marfim e usando luvas de pelica; eu não creio que Deus
abençoe esse trabalho. Embrenhem-se no campo de trigo, como homens determinados! Os servos
de Deus deveriam sentir que são um com o povo; sejam quem forem as pessoas, os servos
deveriam amá-las, relacionar-se com elas, sentir prazer em vê-las. Olhar para essas pessoas e
dizer: “irmão”. Todo homem é meu irmão, mesmo o mau, porém eu o amarei e o levarei até
Jesus. Os ceifeiros de Cristo devem descer até o trigo.
Agora, observe o que o trabalhador traz consigo: uma foice. Sua comunicação com a espiga de
trigo é rápida e afiada. Ele a corta; corta, e a espiga cai ao chão. O homem que Deus diz ser um
trabalhador da seara não pode usar de palavras suaves e delicadas, persuadir com doutrinas que
falam da dignidade humana, da excelência da autoajuda, do esforço próprio para corrigir a
condição pecaminosa e coisas semelhantes. Deus pode amaldiçoar tal hipócrita, pois ele é a
maldição deste século. O pregador honesto chama o pecado de pecado e fala francamente: “Você
está se arruinando; enquanto rejeitar Cristo, viverá nas bordas do inferno e brevemente estará
perdido por toda a eternidade. Não há discussão: você tem de escapar da ira divina pela fé em
Jesus, ou será tirado eternamente da presença de Deus e de toda esperança de alegria”. O sermão
do pregador deve ser direto. Ele não tem de cegar a navalha da segadeira temendo machucar
alguém. O evangelho pretende ferir a consciência e ir direto ao coração a fim de separar a alma
do pecado e do ego, da mesma maneira que o grão é separado do solo. Nosso objetivo é cortar
imediatamente o pecado, pois toda a beleza da carne deve ser derrotada; toda a sua glória e
excelência devem ser extirpadas. E o homem deve surgir como um morto para que seja salvo. Os
ministros que não têm o objetivo de cortar profundamente o pecado não são sal da terra. Deus
nunca enviaria homens que não têm como objetivo incomodar a consciência humana. Tal homem

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 38}

**Score:** 349.85894775390625

**Fonte:** O mesmo havia acontecido a Sa vonarola, que abalou a cidade de 
Florença, na Itália, a ponto de o rosto desse “monge louco” tornar -se 
um terror para os florentinos, e motivo de chacota para os fanáticos 
papistas.  
Irmãos, à luz do conhecimento que temos sobre o altar de Deus, é 
melhor vi vermos seis meses com o coração em chamas, apontando o 
pecado deste mundo, seja em que lugar for, e conclamando o povo a 
libertar -se do poder de Satanás e se voltar para Deus (como fez João 
Batista), do que morrer cercado de honrarias eclesiásticas e de 
doutorados em teologia, para se tornar motivo de riso no inferno, para 
os espíritos das trevas. Ridicularizar os magnatas da bebida e 
censurar os políticos corruptos não atrai maldição sobre a nossa 
cabeça. Há quem faça as duas coisas sem sofrer nenhuma amea ça à 
sua vida e à sua posição no púlpito. Os profetas do passado eram 
mortos po rque combatiam veementemente as religiões falsas. E nós 
também devíamos nos deixar arder de santa indignação ao ver a 
religião falsa enganando nossos semelhantes, e roubando de nossos 
entes queridos a salvação; ou ao ver sacerdotes levando -os para o 
inferno sob a efígie de um crucifixo. E talvez, quem sabe, daqui a 
alguns anos, p ara abrir o caminho para uma nova reforma no século 
XX, nós sejamos queimados em fogueiras.  
Esta aqui é para se ler e chorar: “Hoje o protestantismo mutilado vê 
os sacerdotes católicos romanos elogiando os evangelistas 
protestantes”. Respond a em sã consciência, você acredita que esses 
mesmos papistas aplaudiriam Lutero ou apoiariam Savo naro la? Ó 
Deus, envi a-nos pregadores que possam entre gar mensagens que 
penetrem o coração dos homens e o incendeie! Envia -nos uma 
geração de pregadores mártires, de homens em chamas, 
quebrantados e prostrados diante da visão de um castigo iminente e 
de um inferno eterno para os irregenerados!  
Que Deus nos envie profetas, homens destemidos que clamem em 
alta voz e não poupem ninguém, que  abalem nações com lamentos 
ungidos, que sejam fer vorosos quase a ponto de se tornarem 
insuportáveis, duros a ponto de ser difícil ouvi -los, e 
descomprome tidos a ponto de sofrerem perseguição. Esta mos 
cansados de pregadores que se apresentam de roupas elegantes, 
linguagem suave e torrentes de palavras, mas apenas com uma gota 
de unção. São homens que entendem mais de competição do que 
consagração , mais de promoção d o que oração. Substituem o

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 72}

...

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Score:** 365.09185791015625

**Fonte:** inferno? Quem irá r esistir ao diabo? Quem quer privar -se de alimento, 
descanso e lazer, para que os inf ernos o vejam lutando, 
envergonhando os demônios, libertando os cativos, esvaziando o 
inferno, e sofrendo as dores de parto para deixar atrás de si uma 
fileira de pessoas lavadas pelo sa ngue de Cristo?  
Em último lugar, o avivamento tarda porque rou bamos a glória que 
pertence a Deus . Reflitamos um pouco sobre essas palavras de 
Jesus: “Eu não  aceito glória que vem dos homens”. “Como podeis 
crer, vós os que aceitais glória uns dos outros, e contudo não 
procurais a glória que vem do Deus único?” (Jo 5.41,44.) C hega de 
toda essa autopromoção nos púlpitos. Chega de tanto exaltar “ meu 
programa de rádio”, “ minha  igreja”, “meus  livros”. Ah, que repulsiva 
demonstração carnal vemos nos púlpitos: “Hoje, temos o grande 
privilégio...” E os pregadores aceitam isso; não, el es já o esperam. (E 
se esquecem de que só estão ali pela graça de Deus .) E a vaidade é 
que, quando ouvimos tais homens pregar, notamos que nunca 
ficaríamos sabendo que eram tão importantes, se não tivessem sido 
apresentados como tal.  
Coitado de Deus!  Ele n ão está recebendo muita glória! Então, por 
que ele ainda não cumpriu sua terrível mas bendita ameaça de que 
iria vomitar -nos de sua boca? Nós fracassamos; estamos impuros. 
Apreciamos os louvores dos homens. Buscamos nossos próprios 
interesses. Ó Deus, libe rta-nos dessa existência egoística, 
egocêntrica! Dá -nos a bênção do quebrantamento! O juízo deve 
começar por nós, pelos pregadores!

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 42}

**Score:** 375.86114501953125